In [ ]:
# The purpose of this batch notebook is to manually label the grid_bin_bounds for each session using the interactive GUI.
# specific_session_override_dict = { 
#     # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'):{'grid_bin_bounds':(((29.16, 261.7), (133.87292045454544, 150.19888636363635)))},
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'):{'grid_bin_bounds':((29.16, 261.70), (130.23, 150.99))},
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'):{'grid_bin_bounds':((22.397021260868584, 245.6584673739576), (133.66465594522782, 155.97244934208123))},
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'):{'grid_bin_bounds':((28.36, 244.82), (138.74, 156.39))},
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'):{'grid_bin_bounds':(((29.088604852961407, 251.70402561515647), (138.496638485457, 154.30675703402517)))},
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'):{'grid_bin_bounds':((28.54313873072426, 255.54313873072425), (-56.2405385510412, -12.237798967230454))},
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'):{'grid_bin_bounds':((25.5637332724328, 257.964172947664), (89.1844223602494, 131.92462510535915))},
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'):{'grid_bin_bounds':(((29.64642522460817, 257.8732552112081), (106.68603845428224, 146.71219371189815)))},
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'):{'grid_bin_bounds':(((36.47611374385336, 246.658598426423), (134.75608863422366, 149.10512838805013)))},
#     # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'):{'grid_bin_bounds':(((19.639345624112345, 248.63934562411234), (134.21607306829767, 154.57926689187622)))},
# }

In [1]:

%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3
from pathlib import Path
import pathlib
import numpy as np
import pandas as pd

# required to enable non-blocking interaction:
%gui qt5

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.function_helpers import function_attributes

# pyPhoPlaceCellAnalysis:
# NeuroPy (Diba Lab Python Repo) Loading
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass

## For computation parameters:
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths

# from PendingNotebookCode import _perform_batch_plot, _build_batch_plot_kwargs
# from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_load_session, batch_extended_computations, SessionBatchProgress, batch_programmatic_figures, batch_extended_programmatic_figures
# from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData, loadData
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme
from pyphoplacecellanalysis.General.Batch.runBatch import BatchRun
from pyphoplacecellanalysis.General.Batch.runBatch import run_diba_batch

from scripts.run_BatchAnalysis import post_compute_validate, _on_complete_success_execution_session
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots


active_global_batch_result_filename='global_batch_result_grid_bin_bounds.pkl'
debug_print=True

Automatic pdb calling has been turned OFF
build_module_logger(module_name="Spike3D.pipeline"):
	 Module logger com.PhoHale.Spike3D.pipeline has file logging enabled and will log to EXTERNAL/TESTING/Logging/debug_com.PhoHale.Spike3D.pipeline.log


In [2]:
""" 
from pyphoplacecellanalysis.General.Batch.runBatch import main, BatchRun, run_diba_batch, run_specific_batch

"""
global_data_root_parent_path = find_first_extant_path([Path(r'W:\Data'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

## Build Pickle Path:
global_batch_result_file_path = Path(global_data_root_parent_path).joinpath(active_global_batch_result_filename).resolve() # Use Default

# try to load an existing batch result:
global_batch_run = BatchRun.try_init_from_file(global_data_root_parent_path, active_global_batch_result_filename=active_global_batch_result_filename, debug_print=debug_print) # on_needs_create_callback_fn=run_diba_batch
batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=False) # all
good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)
batch_progress_df

finalized_loaded_global_batch_result_pickle_path: /media/MAX/Data/global_batch_result_grid_bin_bounds.pkl
Loading loaded session pickle file results : /media/MAX/Data/global_batch_result_grid_bin_bounds.pkl... encountered exception 'BatchRun' object is not iterable while printing. Turning into a warning and continuing.
done.
no difference between provided and internal paths.


,format_name,animal,exper_name,session_name,context,basedirs,status,errors,n_long_laps,n_long_replays,n_short_laps,n_short_replays,is_ready
0,kdiba,gor01,one,2006-6-07_11-26-53,kdiba_gor01_one_2006-6-07_11-26-53,/media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-2...,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False
1,kdiba,gor01,one,2006-6-08_14-26-15,kdiba_gor01_one_2006-6-08_14-26-15,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False
2,kdiba,gor01,one,2006-6-09_1-22-43,kdiba_gor01_one_2006-6-09_1-22-43,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22-43,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False
3,kdiba,gor01,one,2006-6-09_3-23-37,kdiba_gor01_one_2006-6-09_3-23-37,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_3-23-37,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False
4,kdiba,gor01,one,2006-6-12_15-55-31,kdiba_gor01_one_2006-6-12_15-55-31,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,kdiba,pin01,one,one,kdiba_pin01_one_one,/media/MAX/Data/KDIBA/pin01,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False
69,kdiba,pin01,one,redundant,kdiba_pin01_one_redundant,/media/MAX/Data/KDIBA/pin01/redundant,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False
70,kdiba,pin01,one,showclus,kdiba_pin01_one_showclus,/media/MAX/Data/KDIBA/pin01/showclus,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False
71,kdiba,pin01,one,sleep,kdiba_pin01_one_sleep,/media/MAX/Data/KDIBA/pin01/sleep,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False


# Build `global_batch_run` pre-loading results (before execution)

In [ ]:

# global_batch_result = loadData('global_batch_result.pkl')
global_batch_run = run_diba_batch(global_data_root_parent_path, execute_all=False, extant_batch_run=global_batch_run, debug_print=False)
# print(f'global_batch_result: {global_batch_run}')
global_batch_run

# Run Batch Executions/Computations

### Session Complete Callback Function

In [3]:
specific_session_override_dict = { 
    # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'):{'grid_bin_bounds':(((29.16, 261.7), (133.87292045454544, 150.19888636363635)))},
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'):{'grid_bin_bounds':((29.16, 261.70), (130.23, 150.99))},
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'):{'grid_bin_bounds':((22.397021260868584, 245.6584673739576), (133.66465594522782, 155.97244934208123))},
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'):{'grid_bin_bounds':((28.36, 244.82), (138.74, 156.39))},
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'):{'grid_bin_bounds':(((29.088604852961407, 251.70402561515647), (138.496638485457, 154.30675703402517)))},
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'):{'grid_bin_bounds':(((29.16, 261.7), (133.87292045454544, 150.19888636363635)))},
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'):{'grid_bin_bounds':((25.5637332724328, 257.964172947664), (89.1844223602494, 131.92462510535915))},
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'):{'grid_bin_bounds':(((19.639345624112345, 248.63934562411234), (134.21607306829767, 154.57926689187622)))},
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'):{'grid_bin_bounds':((28.54313873072426, 255.54313873072425), (-56.2405385510412, -12.237798967230454))},
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'):{'grid_bin_bounds':((25.5637332724328, 257.964172947664), (89.1844223602494, 131.92462510535915))},
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_3-23-37'):{'grid_bin_bounds':(((29.64642522460817, 257.8732552112081), (106.68603845428224, 146.71219371189815)))},
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'):{'grid_bin_bounds':(((36.47611374385336, 246.658598426423), (134.75608863422366, 149.10512838805013)))},
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6'):{'grid_bin_bounds':(((34.889907585004366, 250.88049171752402), (131.38802948402946, 148.80548955773958)))},
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_15-46-47'):{'grid_bin_bounds':(((37.58127153781621, 248.7032779553949), (133.5550653393467, 147.88514770982718)))},
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-17_12-33-47'):{'grid_bin_bounds':(((26.23480758754316, 249.30607830191923), (130.58181353748455, 153.36300919999059)))},
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-18_13-6-1'):{'grid_bin_bounds':(((31.470464455344967, 252.05028043482017), (128.05945067500747, 150.3229156741395)))},
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_13-34-40'):{'grid_bin_bounds':(((29.637787747400818, 244.6377877474008), (138.47834488369824, 155.0993015545914)))},
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-27_14-43-12'):{'grid_bin_bounds':(((27.16098236570231, 249.70986567911666), (106.81005068995495, 118.74413456592755)))},
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'):{'grid_bin_bounds':(((28.84138997640293, 259.56043988873074), (101.90256273413083, 118.33845994931318)))},
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-11_12-48-38'):{'grid_bin_bounds':(((21.01014932647431, 250.0101493264743), (92.34934413366932, 128.1552287735411)))},
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-11_16-2-46'):{'grid_bin_bounds':(((17.270839996578303, 259.97986762679335), (94.26725170377283, 131.3621243061284)))},
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_15-25-59'):{'grid_bin_bounds':(((30.511181558838498, 247.5111815588389), (106.97411662767412, 146.12444016982818)))},
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_14-49-24'):{'grid_bin_bounds':(((30.473731136762368, 250.59478046470133), (105.10585244511995, 149.36442051808177)))},
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52'):{'grid_bin_bounds':(((27.439671363238585, 252.43967136323857), (106.37372678405141, 149.37372678405143)))},
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15'):{'grid_bin_bounds':(((25.118453388111003, 253.3770388211908), (106.67602982073078, 145.67602982073078)))},
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-19_13-50-7'):{'grid_bin_bounds':(((22.47237613669028, 247.4723761366903), (109.8597911774777, 148.96242871522395)))},
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-19_16-37-40'):{'grid_bin_bounds':(((27.10059856429566, 249.16997904433555), (104.99819196992492, 148.0743732909197)))},
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-21_11-19-2'):{'grid_bin_bounds':(((19.0172498755827, 255.42277198494864), (110.04725120825609, 146.9523233129975)))},
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55'):{'grid_bin_bounds':(((12.844282158261015, 249.81408485606906), (107.18107171696062, 147.5733884981106)))},
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-26_13-51-50'):{'grid_bin_bounds':(((29.04362374788327, 248.04362374788326), (104.87398380095135, 145.87398380095135)))},
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13'):{'grid_bin_bounds':(((14.219834349211556, 256.8892365192059), (104.62582591329034, 144.76901436952045)))},
}
# alternatives: IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'):{'grid_bin_bounds':(((12.306345490551518, 255.3063454905515), (106.38370032931314, 147.09201856289755)))},

output_dict_lines = []

In [5]:
import matplotlib
%matplotlib qt
from neuropy.utils.matplotlib_helpers import interactive_select_grid_bin_bounds_2D

def _custom_on_complete_success_execution_session(active_batch_run: BatchRun, curr_session_context, curr_session_basedir, curr_active_pipeline):
    """ called when the execute_session completes like:
        `post_run_callback_fn_output = post_run_callback_fn(curr_session_context, curr_session_basedir, curr_active_pipeline)`
        
        Meant to be assigned like:
        , post_run_callback_fn=_on_complete_success_execution_session
        
        Captures `specific_session_override_dict`.
        
        from Spike3D.scripts.run_BatchAnalysis import _on_complete_success_execution_session
        
    """
    print(f'_custom_on_complete_success_execution_session(curr_session_context: {curr_session_context}, curr_session_basedir: {str(curr_session_basedir)}, ...)')
    # print(f'curr_session_context: {curr_session_context}, curr_session_basedir: {str(curr_session_basedir)}')
    # long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
    # long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
    # long_results, short_results, global_results = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]

    global_epoch_name = 'maze'
    
    # ## Post Compute Validate 2023-05-16:
    # post_compute_validate(curr_active_pipeline)    

    # Iteractively allow selection of the grid_bin_bounds:
    grid_bin_bounds_extents = interactive_select_grid_bin_bounds_2D(curr_active_pipeline, epoch_name=global_epoch_name, should_block_for_input=True)
    # x0, x1, y0, y1 = grid_bin_bounds_extents
    # grid_bin_bounds = ((x0, x1), (y0, y1))
    print(f'curr_session_context: {curr_session_context}, grid_bin_bounds: {grid_bin_bounds_extents}')
    x0, x1, y0, y1 = grid_bin_bounds_extents
    specific_session_override_dict[curr_session_context] = ((x0, x1), (y0, y1))
    out_dict_defn_line = f"{curr_session_context.get_initialization_code_string()}:dict(grid_bin_bounds=({((x0, x1), (y0, y1))}))"
    print(f"\t{out_dict_defn_line}\n")
    output_dict_lines.append(out_dict_defn_line)
    # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'):{'grid_bin_bounds':((25.5637332724328, 257.964172947664), (89.1844223602494, 131.92462510535915))},
    return None
    # return {long_epoch_name:(long_laps, long_replays), short_epoch_name:(short_laps, short_replays),
    #         'outputs': {'local': curr_active_pipeline.pickle_path,
    #                     'global': curr_active_pipeline.global_computation_results_pickle_path}
    #     }

## I got it doing the bare-minimum loading and computations, so it should be ready to update the laps and constraint the placefields to those. Then we should be able to set up the replays at the same time.
# finally, we then finish by computing.
# force_reload = True



force_reload = False
saving_mode = PipelineSavingScheme.SKIP_SAVING
## Execute with the custom arguments.
active_computation_functions_name_includelist=['_perform_baseline_placefield_computation']
epoch_name_includelist = ['maze']
# epoch_name_includelist = None
# active_computation_functions_name_includelist=['_perform_baseline_placefield_computation']
# global_batch_run.execute_all(skip_extended_batch_computations=True, post_run_callback_fn=_custom_on_complete_success_execution_session,
#                                   force_reload = force_reload,
#                               **{'computation_functions_name_includelist': active_computation_functions_name_includelist,
#                                  'active_session_computation_configs': None,
#                                  'saving_mode': saving_mode,
#                                  'epoch_name_includelist': epoch_name_includelist}) 


# global_batch_run.execute_session(IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'),
#                                   skip_extended_batch_computations=True, post_run_callback_fn=_custom_on_complete_success_execution_session,
#                                   force_reload = force_reload,
#                               **{'computation_functions_name_includelist': active_computation_functions_name_includelist,
#                                  'active_session_computation_configs': None,
#                                  'saving_mode': saving_mode,
#                                  'epoch_name_includelist': epoch_name_includelist})


for a_ctxt in global_batch_run.session_contexts:
    # if a_ctxt not in specific_session_override_dict:
    global_batch_run.execute_session(a_ctxt, skip_extended_batch_computations=True, post_run_callback_fn=_custom_on_complete_success_execution_session,
                                force_reload = force_reload,
                                fail_on_exception = True,
                            **{'computation_functions_name_includelist': active_computation_functions_name_includelist,
                                'active_session_computation_configs': None,
                                'saving_mode': saving_mode,
                                'epoch_name_includelist': epoch_name_includelist})


specific_session_override_dict
output_dict_lines
# 4m 39.8s

basedir: /media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-26-53
active_data_mode_name: kdiba
Loading loaded session pickle file results : /media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-26-53/loadedSessPickle.pkl... Issue with pickled WindowsPath on Linux for path /media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-26-53/loadedSessPickle.pkl, performing pathlib workaround...
done.
basedir: /media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-26-15
active_data_mode_name: kdiba
Loading loaded session pickle file results : /media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-26-15/loadedSessPickle.pkl... done.
Loading pickled pipeline success: /media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-26-15/loadedSessPickle.pkl.
property already present in pickled version. No need to save.
using provided computation_functions_name_includelist: ['_perform_baseline_placefield_computation']
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is s

qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


user confirmed extents: (29.16, 261.7, 130.23, 150.99)
_on_update_grid_bin_bounds(new_grid_bin_bounds: (29.16, 261.7, 130.23, 150.99))
Add this to `specific_session_override_dict`:

IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'):dict(grid_bin_bounds=(((29.16, 261.7), (130.23, 150.99)))),

curr_session_context: kdiba_gor01_one_2006-6-08_14-26-15, grid_bin_bounds: (29.16, 261.7, 130.23, 150.99)


ValueError: Invalid format specifier

In [ ]:
print(',\n'.join(output_dict_lines))

In [ ]:
global_batch_run.to_dataframe()

In [ ]:
# last completed:
r'W:\Data\KDIBA\pin01\one\11-03_12-3-25'

In [ ]:
# Save to file:
saveData(global_batch_result_file_path, global_batch_run) # Update the global batch run dictionary


In [ ]:
# Get output files:
batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=False) # all
good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)
good_only_batch_progress_df

In [ ]:
## Build a list of the output files for the good sessions:
session_result_paths = [str(v.joinpath(f'loadedSessPickle.pkl').resolve()) for v in list(good_only_batch_progress_df.basedirs.values)]
global_computation_result_paths = [str(v.joinpath(f'output/global_computation_results.pkl').resolve()) for v in list(good_only_batch_progress_df.basedirs.values)]

# Write out a GreatlakesOutputs.txt file:
with open('GreatlakesOutputs.txt','w') as f:
    f.write('\n'.join(session_result_paths + global_computation_result_paths))
    # f.write('\n'.join())


In [ ]:
# good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=False, good_only=True)
# good_only_batch_progress_df

[v.get('outputs', None) for v in list(global_batch_run.session_batch_outputs.values()) if v is not None]
# v = list(global_batch_run.session_batch_outputs.values())[0]
# v.get('outputs', None)
  

# [{'local': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-07_11-26-53/loadedSessPickle.pkl'),
#   'global': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-07_11-26-53/output/global_computation_results.pkl')},
#  {'local': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-08_14-26-15/loadedSessPickle.pkl'),
#   'global': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_results.pkl')},
#  ...
# ]
    


# outputs': {'local': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-08_14-26-15/loadedSessPickle.pkl'),
#    'global': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_results.pkl')}}

## Single Session testing:


In [ ]:
_test_out = global_batch_run.execute_session(session_context=curr_sess_context, force_reload=True, skip_extended_batch_computations=True, computation_functions_name_includelist =['_perform_baseline_placefield_computation'], active_session_computation_configs=None) # can override `active_session_computation_configs` if we want to set custom ones like only the laps.)
_test_out

# global_batch_run.execute_session(session_context=curr_sess_context, force_reload=True, skip_extended_batch_computations=True, **{'computation_functions_name_includelist': ['_perform_baseline_placefield_computation'], 'active_session_computation_configs': None}) # can override `active_session_computation_configs` if we want to set custom ones like only the laps.)

# 23.5s

In [ ]:

full_good_dirs = [k for k, v in global_batch_run.session_batch_errors.items() if v is None]
bad_dirs = [k for k, v in global_batch_run.session_batch_errors.items() if v is not None]
full_good_dirs
bad_dirs

In [ ]:
global_batch_run.session_batch_status

In [ ]:
global_batch_run.session_batch_status
global_batch_run.session_batch_errors

# Get good sessions for use in the specific session processing notebook:

In [ ]:
batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=False) # all
good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)
good_only_batch_progress_df

In [ ]:
## Get the list of sessions that are completely ready to process:
full_good_ready_to_process_sessions = list(good_only_batch_progress_df['context'].to_numpy())
full_good_ready_to_process_sessions
# Get good sessions for use in the specific session processing notebook:

In [ ]:
run["good_sessions_list"].extend(full_good_ready_to_process_sessions)

In [ ]:
run.stop()
project.stop()

In [ ]:

print(",\n".join([ctx.get_initialization_code_string() for ctx in full_good_ready_to_process_sessions])) # List definitions

# [IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
# IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30')]

In [ ]:
print("\ncurr_context = ".join([ctx.get_initialization_code_string() for ctx in full_good_ready_to_process_sessions])) # Line definitions

# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30')

In [ ]:
good_only_batch_progress_df

In [ ]:
from datetime import datetime

# datetime object containing current date and time
save_time = datetime.now()
 
print("save_time =", save_time)

# dd/mm/YY H:M:S
dt_string = save_time.strftime("%Y-%m-%d_%I-%M%p")
print("date and time =", dt_string)

In [ ]:
## Get output file paths:
completed_pipeline_filename = 'loadedSessPickle.pkl'
completed_global_computations_filename = 'outputs/global_computation_results.pkl'

full_good_ready_to_process_session_paths = list(good_only_batch_progress_df['basedirs'].to_numpy())
session_paths_output_folders = [sess_path.joinpath('outputs').resolve() for sess_path in full_good_ready_to_process_session_paths]



completed_pipeline_file_paths = [sess_path.joinpath(completed_pipeline_filename).resolve() for sess_path in full_good_ready_to_process_session_paths]
completed_global_computations_file_paths = [sess_path.joinpath(completed_global_computations_filename).resolve() for sess_path in full_good_ready_to_process_session_paths]
completed_global_computations_file_paths